# 데이터 불러오기

In [ ]:
import os
import shutil
import yaml
from ultralytics import YOLO


In [ ]:
# PPE-1 데이터셋 저장 경로
ppe1_dataset = './filtered_PPE-1'
ppe1_yaml = os.path.join(ppe1_dataset, 'data.yaml')

# PPE-2 데이터셋 저장 경로
ppe2_dataset = './filtered_PPE-2'
ppe2_yaml = os.path.join(ppe2_dataset, 'data.yaml')

# 모델 학습

## PPE-1 데이터셋만 학습

In [ ]:
model = YOLO('yolo11n.pt') # v8 사용하고 싶다면 yolov8n.pt로 작성

results = model.train(
    data=ppe1_yaml,
    epochs=100,
    imgsz=640,
    batch=16, # 본인 gpu 성능에 맞게 수정 (작으면 정밀하지만 느리게 학습 / 크면 반대)
    workers=2, # 본인 cpu 코어 맞게 수정
    name='ppe1_detection',
    patience=10, # 10번 학습할 동안 성능 개선 없으면 학습 종료
    save=True, # 모델 가중치 자동 저장 best.pt, last.pt
    plots=True # 학습 분석 가능한 다양한 그래프 생성 (loss curves 등)
)


## PPE-2 데이터셋 추가 학습

In [ ]:
model = YOLO('./runs/detect/ppe1_detection/weights/best.pt')

results = model.train(
    data=ppe2_yaml,
    epochs=100,
    imgsz=640,
    batch=16, # 본인 gpu 성능에 맞게 수정 (작으면 정밀하지만 느리게 학습 / 크면 반대)
    workers=2, # 본인 cpu 코어 맞게 수정
    name='ppe1_2_detection',
    patience=30, # 10번 학습할 동안 성능 개선 없으면 학습 종료
    save=True, # 모델 가중치 자동 저장 best.pt, last.pt
    plots=True # 학습 분석 가능한 다양한 그래프 생성 (loss curves 등)
)


# 모델 성능 평가

In [ ]:
import pandas as pd

df = pd.read_csv('runs/detect/ppe1_2_detection/results.csv') # 보고싶은 모델 results.csv 경로로 지정
df.tail(1)  # 마지막 행 전체 보기


# 테스트

In [ ]:
test_img_dir ='final_dataset/test/images'
model = YOLO('./runs/detect/ppe1_detection/weights/best.pt')

results = model.predict(
    source=test_img_dir,
    save=True, # 결과 자동 저장
    conf=0.25, # 정확도 25% 넘으면 다 검출
    project='runs/detect', # 결과 저장할 경로
    name='ppe1_test', # 이름은 맘대로 지정
    exist_ok=False # True : 같은 이름의 폴더가 이미 존재하면 덮어쓰기 False : 같은 이름 있으면 뒷번호로 새로 만들기
)


In [ ]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
from PIL import Image
result_img_dir = 'runs/detect/ppe1_test' # 보고 싶은 결과 경로 맞는지 확인하기

# 테스트 이미지 불러오기
test_images = glob.glob(os.path.join(result_img_dir, '*.jpg'))[220:] # 몇 개 불러올 건지 결정

# 그리드 설정
n_cols = 4 # 행 개수 지정
n_rows = (len(test_images) + n_cols - 1) // n_cols # 열 자동 설정

# Figure 생성
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4 * n_rows))
axes = axes.flatten()

# 표시
for idx, img_path in enumerate(test_images):
    img = Image.open(img_path)
    axes[idx].imshow(img)
    axes[idx].axis('off')

# 빈 칸 제거
for idx in range(len(test_images), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()
